In [2]:
!pip install --upgrade s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.7/246.7 kB 6.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.8/365.8 kB 7.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.40.52
    Uninstalling botocore-1.40.52:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'RECORD'
Consider using the `--user` option or check the

In [3]:
!pip install dlt
!pip install minio
!pip install --upgrade pyarrow
!pip install --upgrade dlt
!pip install --user "dlt[s3]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.2 MB/s eta 0:00:00a 0:00:01
  Using cached s3fs-2025.9.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached aiobotocore-2.25.0-py3-none-any.whl.metadata (25 kB)
  Using cached aiohttp-3.13.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached botocore-1.40.49-py3-none-any.whl.metadata (5.7 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
Using cached s3fs-2025.9.0-py3-none-any.whl (30 kB)
Using cached aiobotocore-2.25.0-py3-none-any.whl (86 kB)
Using cached botocore-1.40.49-py3-none-any.whl (14.1 MB)
Using cached aiohttp-3.13.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (1.7 MB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [3]:
import boto3
from botocore.exceptions import EndpointConnectionError, ClientError

# Aquí define tus credenciales para revisar si conecta Jupyter con Minio. ¡Importante hacer esto para estar seguro de tus rutas!
ENDPOINT = "http://minio:9000"
ACCESS   = "minioadmin"
SECRET   = "minioadmin"

try:
    s3 = boto3.client("s3",
                      endpoint_url=ENDPOINT,
                      aws_access_key_id=ACCESS,
                      aws_secret_access_key=SECRET)
    response = s3.list_buckets()
    response = s3.list_buckets()
    print("Conectado correctamente a MinIO")
    print("Buckets encontrados:", [b['Name'] for b in response.get('Buckets', [])])
except EndpointConnectionError as e:
    print("¡ERROR! No se pudo conectar al endpoint:", e)
except ClientError as e:
    print("¡OJO! Error de cliente S3:", e)
except Exception as e:
    print("Otro error:", e)


Conectado correctamente a MinIO
Buckets encontrados: ['lakehouse', 'mybucket']


In [2]:
import os
import boto3
from botocore.exceptions import ClientError

s3 = boto3.client(
    "s3",
    endpoint_url="http://minio:9000", 
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin"
)

bucket = "lakehouse"
base_path = "/data/StackOverflowData"  # carpeta dentro DEL CONTENEDOR JUPYTER

# Verificación por si la ruta no exite.
if not os.path.exists(base_path):
    raise FileNotFoundError(f"No se encuentra la ruta {base_path}. Verifica el volumen en docker-compose.")

# Crear bronze por si no existe.
prefix = "bronze/"
try:
    s3.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=1)
except ClientError as e:
    print("Error al acceder al bucket:", e)

# Subir todo lo que sea .parquet! AVISO: Tarda su tiempo todo depende de los parquets seleccionados.
EXCLUIR = {"comments_2021.parquet"} 

files = [f for f in os.listdir(base_path) if f.endswith(".parquet")]
print(f"Encontrados {len(files)} archivos parquet en {base_path}")

for i, file in enumerate(files, 1):
    if file in EXCLUIR:
        print(f"[{i}/{len(files)}] Saltando {file} (se cargará con DLT)")
        continue  # pasa al siguiente archivo

    src = os.path.join(base_path, file)
    dest = f"{prefix}{file}"
    print(f"[{i}/{len(files)}] Subiendo {file} a MinIO ...", end=" ")
    s3.upload_file(src, bucket, dest)
    print("LISTO!")

print("Ingesta manual Bronze completada.")

Encontrados 5 archivos parquet en /data/StackOverflowData
[1/5] Subiendo badges.parquet a MinIO ... LISTO!
[2/5] Subiendo comments_2020.parquet a MinIO ... LISTO!
[3/5] Subiendo posts_2020.parquet a MinIO ... LISTO!
[4/5] Subiendo posts_2021.parquet a MinIO ... LISTO!
[5/5] Subiendo users.parquet a MinIO ... LISTO!
Ingesta manual Bronze completada.


In [2]:
import os
import sys
import time
import tempfile
from pathlib import Path
from typing import Iterator, Optional

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import pyarrow as pa
import pyarrow.parquet as pq

import dlt

import boto3
from botocore.config import Config as BotoConfig
from boto3.s3.transfer import TransferConfig
from botocore.exceptions import ClientError


# ---------- Config ----------
URL = "https://datasets-documentation.s3.eu-west-3.amazonaws.com/stackoverflow/parquet/comments/2021.parquet"

# MinIO (ajusta si corres fuera del contenedor)
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "http://minio:9000")
MINIO_ACCESS   = os.environ.get("MINIO_ACCESS_KEY_ID", "minioadmin")
MINIO_SECRET   = os.environ.get("MINIO_SECRET_ACCESS_KEY", "minioadmin")
BUCKET         = os.environ.get("MINIO_BUCKET", "lakehouse")
PREFIX         = os.environ.get("MINIO_PREFIX", "bronze")  # destino lógico en Bronze
FINAL_PREFIX   = f"{PREFIX.rstrip('/')}/comments/2021/"

# DLT: rutas y opciones
PIPELINE_NAME  = "bronze_ingest_dlt_comments2021"
DATASET_NAME   = "comments_2021"      # nombre lógico en filesystem
PIPELINES_DIR  = "/workspace/.dlt"    # estado local de dlt

# Rendimiento / robustez
HTTP_TIMEOUT_S = (10, 60)  # (connect, read)
HTTP_RETRIES   = 5
HTTP_BACKOFF   = 0.5
STREAM_CHUNK_MB = 8                     # chunk de descarga
ARROW_BATCH_ROWS = 200_000              # tamaño aproximado del batch (ajusta según RAM)
S3_MAX_CONCURRENCY = int(os.environ.get("S3_MAX_CONCURRENCY", "8"))
S3_MULTIPART_CHUNK_MB = int(os.environ.get("S3_MULTIPART_CHUNK_MB", "16"))
S3_RETRIES     = int(os.environ.get("S3_RETRIES", "5"))


# ---------- Utilidades ----------
def _requests_session() -> requests.Session:
    s = requests.Session()
    retries = Retry(
        total=HTTP_RETRIES,
        connect=HTTP_RETRIES,
        read=HTTP_RETRIES,
        backoff_factor=HTTP_BACKOFF,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retries, pool_connections=4, pool_maxsize=8)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    return s


def _human(n: int) -> str:
    for unit in ("", "K", "M", "B"):
        if abs(n) < 1000:
            return f"{n:.0f}{unit}"
        n /= 1000
    return f"{n:.0f}T"


# ---------- Resource DLT ----------
@dlt.resource(name="comments_2021", write_disposition="replace")
def comments_2021_arrow() -> Iterator[pa.Table]:
    """
    Descarga el Parquet remoto de forma resiliente y lo entrega
    como una secuencia de Arrow Tables en batches para limitar memoria.
    """
    print(f"[DL] Descargando: {URL}")
    session = _requests_session()
    with session.get(URL, stream=True, timeout=HTTP_TIMEOUT_S) as r:
        r.raise_for_status()
        with tempfile.NamedTemporaryFile(suffix=".parquet") as tmp:
            chunk_bytes = STREAM_CHUNK_MB * 1024 * 1024
            total = 0
            t0 = time.time()
            for chunk in r.iter_content(chunk_size=chunk_bytes):
                if chunk:
                    tmp.write(chunk)
                    total += len(chunk)
                    if total % (64 * 1024 * 1024) < chunk_bytes:
                        elapsed = time.time() - t0
                        mb = total / (1024 * 1024)
                        rate = mb / max(elapsed, 1e-6)
                        print(f"[DL] {mb:.1f} MiB ({rate:.1f} MiB/s)")
            tmp.flush()

            # Streaming en lotes con PyArrow (no carga todo en RAM)
            pf = pq.ParquetFile(tmp.name)
            rows_total = 0
            cols = None
            for batch in pf.iter_batches(batch_size=ARROW_BATCH_ROWS):
                tbl = pa.Table.from_batches([batch])
                rows_total += tbl.num_rows
                cols = tbl.num_columns if cols is None else cols
                print(f"[ARROW] Batch → {tbl.num_rows} filas (acum: {_human(rows_total)})")
                yield tbl

            print(f"[ARROW] OK: {_human(rows_total)} filas, {cols} columnas")


# ---------- S3/MinIO ----------
def _s3_client():
    cfg = BotoConfig(
        connect_timeout=15,
        read_timeout=120,
        retries={"max_attempts": S3_RETRIES, "mode": "standard"},
        signature_version="s3v4",
    )
    return boto3.client(
        "s3",
        endpoint_url=MINIO_ENDPOINT,
        aws_access_key_id=MINIO_ACCESS,
        aws_secret_access_key=MINIO_SECRET,
        config=cfg,
    )


def ensure_bucket(s3, bucket: str) -> None:
    try:
        s3.head_bucket(Bucket=bucket)
        print(f"[S3] Bucket '{bucket}' existe.")
    except ClientError as e:
        code = int(e.response.get("ResponseMetadata", {}).get("HTTPStatusCode", 0))
        if code == 404:
            s3.create_bucket(Bucket=bucket)
            print(f"[S3] Bucket '{bucket}' creado.")
        else:
            raise


# ---------- Main ----------
def main() -> None:
    pipeline = dlt.pipeline(
        pipeline_name=PIPELINE_NAME,
        destination="filesystem",     # primero a disco (simple, reproducible)
        dataset_name=DATASET_NAME,
        pipelines_dir=PIPELINES_DIR,
        dev_mode=True,
    )

    print("[DLT] Iniciando ingesta (comments 2021)...")
    load_info = pipeline.run(
        comments_2021_arrow(),
        loader_file_format="parquet",  # fuerza parquet en filesystem
    )
    print("[DLT] Finalizado.")
    print(load_info)

    # Subir a MinIO → lakehouse/bronze/comments/2021/   
    print(f"[OK] Proceso completado. Archivos subidos: {files}")


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] {e}", file=sys.stderr)


[DLT] Iniciando ingesta (comments 2021)...
[DL] Descargando: https://datasets-documentation.s3.eu-west-3.amazonaws.com/stackoverflow/parquet/comments/2021.parquet
[DL] 64.0 MiB (5.0 MiB/s)
[DL] 128.0 MiB (6.2 MiB/s)
[DL] 192.0 MiB (7.1 MiB/s)
[DL] 256.0 MiB (7.8 MiB/s)
[DL] 320.0 MiB (8.2 MiB/s)
[DL] 384.0 MiB (8.5 MiB/s)
[DL] 448.0 MiB (8.8 MiB/s)
[DL] 512.0 MiB (8.8 MiB/s)
[DL] 576.0 MiB (8.8 MiB/s)
[DL] 640.0 MiB (8.9 MiB/s)
[ARROW] Batch → 200000 filas (acum: 200K)
[ARROW] Batch → 200000 filas (acum: 400K)
[ARROW] Batch → 200000 filas (acum: 600K)
[ARROW] Batch → 200000 filas (acum: 800K)
[ARROW] Batch → 200000 filas (acum: 1M)
[ARROW] Batch → 200000 filas (acum: 1M)
[ARROW] Batch → 200000 filas (acum: 1M)
[ARROW] Batch → 200000 filas (acum: 2M)
[ARROW] Batch → 200000 filas (acum: 2M)
[ARROW] Batch → 200000 filas (acum: 2M)
[ARROW] Batch → 200000 filas (acum: 2M)
[ARROW] Batch → 200000 filas (acum: 2M)
[ARROW] Batch → 200000 filas (acum: 3M)
[ARROW] Batch → 200000 filas (acum: 3M)


[ERROR] name 'upload_folder_to_minio' is not defined


In [ ]:
#ANTIGUO
import os
import requests
import tempfile
import pyarrow.parquet as pq
import dlt
import boto3
from botocore.exceptions import ClientError

URL = "https://datasets-documentation.s3.eu-west-3.amazonaws.com/stackoverflow/parquet/comments/2021.parquet"

# MinIO (ajusta si corres fuera del contenedor)
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "http://minio:9000")
MINIO_ACCESS   = os.environ.get("MINIO_ACCESS_KEY_ID", "minioadmin")
MINIO_SECRET   = os.environ.get("MINIO_SECRET_ACCESS_KEY", "minioadmin")
BUCKET         = os.environ.get("MINIO_BUCKET", "lakehouse")
PREFIX         = "bronze"   # destino lógico en Bronze

# DLT: rutas y opciones
PIPELINE_NAME  = "bronze_ingest_dlt_comments2021"
DATASET_NAME   = "comments_2021"          # nombre lógico del dataset en filesystem (no afecta MinIO)
PIPELINES_DIR  = "/workspace/.dlt"      # estado de dlt en disco


@dlt.resource(name="comments_2021", write_disposition="replace")
def comments_2021_arrow():
    """Descarga el Parquet remoto y lo entrega como Arrow Table (streaming seguro)."""
    print(f"Descargando: {URL}")
    with requests.get(URL, stream=True) as r:
        r.raise_for_status()
        with tempfile.NamedTemporaryFile(suffix=".parquet") as tmp:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    tmp.write(chunk)
            tmp.flush()
            table = pq.read_table(tmp.name)  # pyarrow.Table
            print(f"Arrow OK: {table.num_rows} filas, {table.num_columns} columnas")
            # DLT acepta pyarrow.Table; si tu versión no, descomenta el yield de dicts:
            # for rec in table.to_pylist(): yield rec
            yield table


def ensure_bucket(s3, bucket: str):
    try:
        s3.head_bucket(Bucket=bucket)
        print(f"Bucket '{bucket}' existe.")
    except ClientError:
        s3.create_bucket(Bucket=bucket)
        print(f"Bucket '{bucket}' creado.")

def upload_folder_to_minio(local_folder: str, bucket: str, prefix: str):
    """Sube todo el contenido generado por DLT (parquet/metadata) bajo bronze/posts/2021/"""
    s3 = boto3.client(
        "s3",
        endpoint_url=MINIO_ENDPOINT,
        aws_access_key_id=MINIO_ACCESS,
        aws_secret_access_key=MINIO_SECRET,
        config=S3_TIMEOUT_CONFIG,
    )
    ensure_bucket(s3, bucket)


    if not prefix.endswith("/"):
        prefix += "/"
    s3.put_object(Bucket=bucket, Key=prefix)

    uploaded = 0
    for root, _, files in os.walk(local_folder):
        for fname in files:
            local_path = os.path.join(root, fname)
            # clave relativa (conserva estructura de salida de DLT)
            rel = os.path.relpath(local_path, local_folder).replace("\\", "/")
            s3_key = f"{prefix}{rel}"
            print(f"{local_path} → s3://{bucket}/{s3_key}")
            s3.upload_file(local_path, bucket, s3_key)
            uploaded += 1
    print(f"Subidos {uploaded} archivos a s3://{bucket}/{prefix}")


def main():
    pipeline = dlt.pipeline(
        pipeline_name=PIPELINE_NAME,
        destination="filesystem",           # primero a disco
        dataset_name=DATASET_NAME,
        pipelines_dir=PIPELINES_DIR,
        dev_mode=True,
    )

    print("Iniciando ingesta DLT (posts 2021)...")
    load_info = pipeline.run(
        comments_2021_arrow(),
        loader_file_format="parquet",       # fuerza parquet en filesystem
    )
    print("DLT finalizado.")
    print(load_info)

    # 2) Subir a MinIO → lakehouse/bronze/posts/2021/
    output_dir = pipeline.dataset_path()    # carpeta donde DLT escribió parquet
    print(f"Output local DLT: {output_dir}")
    upload_folder_to_minio(output_dir, BUCKET, PREFIX)
    print("Proceso completado.")


if __name__ == "__main__":
    main()


Iniciando ingesta DLT (posts 2021)...
Descargando: https://datasets-documentation.s3.eu-west-3.amazonaws.com/stackoverflow/parquet/comments/2021.parquet
